In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### Custom Functions:

In [ ]:
# TODO: Docstring, function converts "list-like" to string
def unpack_list_like(list_like_series:pd.core.series.Series, asType:str) -> pd.core.series.Series:
    
    # Remove open and close square brackets, single-quotes, and replace commas with forward slashes
    unpacked_series = (list_like_series.str.replace("[","")
                       .str.replace("]","")
                       .str.replace("', '","/")
                       .str.replace("\'",""))
    
    return unpacked_series.astype(asType)

### Section 1: Load and Initial Assesment
In this section, the DataFrame is loaded in raw format in two zipped parts, and concatenated. The method `.info()` of the DataFrame class is used to gather initial insights about the DataFrame.

In [ ]:
# Parse the dataset parts into DataFrames and concatenate them into a single DataFrame
games_sub1 : pd.core.frame.DataFrame = pd.read_csv("datasets/games_may2024_cleaned_1of2.zip", encoding='latin1', low_memory=False)
games_sub2 : pd.core.frame.DataFrame = pd.read_csv("datasets/games_may2024_cleaned_2of2.zip", encoding='latin1', low_memory=False)

games_raw : pd.core.frame.DataFrame = pd.concat([games_sub1, games_sub2])

# Intitial Assessment (info, memory usage, shape, and head)
print("="*20 + " DataFrame Information " + "="*20)
games_raw.info()
print("="*20 + " DataFrame Information " + "="*20)

print("\n" + "="*20 + " Memory Usage " + "="*20)
print(f"{games_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("="*20 + " Memory Usage " + "="*20)

print("\n" + "="*20 + " DataFrame Shape " + "="*20)
print(games_raw.shape)
print("="*20 + " DataFrame Shape " + "="*20)

print("\n" + "="*20 + " DataFrame Head " + "="*20)
print(games_raw.head())
print("="*20 + " DataFrame Head " + "="*20)

### Section 2: Data Quality Assessment
In this section, the data values are examined to inform cleaning decisions.

In [ ]:
# Find the number of NA values in each column
print("\n" + "="*20 + " NA Values" + "="*20)
print(games_raw.isna().sum())
print("="*20 + " NA Values " + "="*20)

# Find the number of unique values in each column
print("\n" + "="*20 + " Unique Values " + "="*20)
print(games_raw.nunique())
print("="*20 + " Unique Values " + "="*20)

### Section 3: Cleaning Decisions
In this section, the DataFrame is cleaned based on the analysis of the previous section, as well as the return of the `.head()` method in Section 1. Section 1 is used to inform type casting decisions, and Section 2 is used to provide early warning of type casting errors.

#### Drop Unwanted Columns:
Columns that do not contribute to analysis of the dataset or aid in answering the question are dropped from the DataFrame in the code cell below.

In [ ]:
# Remove unneeded columns from the dataframe using the .drop() method
games = games_raw.drop(columns=["required_age",
                              "dlc_count",
                              "detailed_description", 
                              "about_the_game", 
                              "short_description", 
                              "reviews", 
                              "support_url", 
                              "support_email", 
                              "estimated_owners",
                              "metacritic_score", 
                              "metacritic_url", 
                              "achievements", 
                              "recommendations", 
                              "notes",
                              "full_audio_languages",
                              "packages",
                              "categories",  
                              "screenshots", 
                              "movies",
                              "user_score", 
                              "score_rank", 
                              "tags",
                              "pct_pos_total",
                              "pct_pos_recent",
                              "average_playtime_forever", 
                              "average_playtime_2weeks", 
                              "median_playtime_forever",
                              "median_playtime_2weeks", 
                              "header_image", 
                              "website"])

#### Set the Index:
In Section 2, it was found that the column "AppID" has _nearly_ the same number of unique values (83653) as the number of rows (83655), making this a great index option. Furtheremore, this column has 0 NA values. For these reasons, "AppID" was selected as the index. Some values were found with clear encoding errors, these were scrapped in the process, as all columns in those rows were improperly encoded, and thus unusable.

In [ ]:
# Set the index:
# Cast the "AppID" column to numeric (NA if not numeric-like) and drop rows with NA values
games['AppID'] = pd.to_numeric(games['AppID'], downcast='integer', errors='coerce')
games = games.dropna(subset=["AppID"])

# Convert remaining rows' "AppID" value to uint32 and then set the index of the DataFrame to this column
games["AppID"] = games['AppID'].astype('uint32')

# Set the data frame index to the "AppID" column
games = games.set_index("AppID")

#### Cast Column Data Types:
Three main types of columns are converted below:
1. Straight-forward string and numeric columns. These are converted to the most appropriate type using `.astype()` with a mapping of column:type pairs as the argument.
2. Columns that are "list-like" (e.g. \['English', 'Vietnamese'\]). These values are modified to be forward slash seperated for subsequent analysis (e.g. English/Vietnamese). Language columns are converted to strings, while genre, developer, and publisher columns are converted to categories due to a high count of repeat values as determined in Section 2.
3. Boolean values. The dataset uses "TRUE" and "FALSE" for its boolean values, which `.astype()` always interprets as True. To solve this problem, each of these columns are initially cast as string, and then are set equal to a boolean mask on the condition `df['col'] == "TRUE"`.

In [ ]:
# Convert straightforward numeric and string column data types
games = games.astype({'name' : 'string',
                      'release_date' : 'datetime64[ns]',
                      'price' : 'float32',
                      'windows' : 'string',
                      'mac' : 'string',
                      'linux' : 'string',
                      'positive' : 'Int64',
                      'negative' : 'Int64',
                      'peak_ccu' : 'Int64',
                      'num_reviews_recent' : 'Int64'
                   })

# Convert the "list-like" columns from ['thing1','thing2'] string to "thing1/thing2" category or string
games["supported_languages"] = unpack_list_like(games["supported_languages"], asType='string')
games['developers'] = unpack_list_like(games["developers"], asType='category')
games['publishers'] = unpack_list_like(games["publishers"], asType='category')
games['genres'] = unpack_list_like(games["genres"], asType='category')

# Set incompatible boolean value columns equal to a bool mask to map TRUE to True and FALSE to False
games['windows'] = games['windows'].str.strip() == "TRUE"
games['mac'] = games['mac'].str.strip() == "TRUE"
games['linux'] = games['linux'].str.strip() == "TRUE"

# Report cleaned DataFrame size
print(f"The size of the cleaned DataFrame is {games.memory_usage(deep=True).sum() / 1024**2:.2f}MB")

# Find the number of unique values in each column
print("\n" + "="*20 + " Unique Values (Cleaned) " + "="*20)
print(games.nunique())
print("="*20 + " Unique Values (Cleaned) " + "="*20)

### Section 4: Statistical EDA
In this section's subsections, several variables/groups of variables are characterized using statistical measurement and visualization transformations. Performing statistical and visual operations on these values allows their distributions to be understood, which provides insight into the measures and their assocaited values. First, individual features are analyzed, and then relations between various features are explored.

### Release Date EDA:
The goal of this section is to characterize the `release_date` column statistically and visually to understand how game release frequencies have changed over time. After cleaning this dataset, 83646 valid observations remain. Section 2 revealed that there are only 4503 unique release dates. As such, it becomes evident that a frequency analysis can provide some insight into the frequency distribution of game release dates. Since there are 4503 unique values of day/month/year, Pandas' built-in plotting struggles to handle axis labels, and as a result, these values were temporarily reduced to a year only value, as this alone is sufficient to understand the change in released game counts over time. Note: Logarithmic scale is used for the number of released games (y-axis) to esnure an insightful bar is plotted for early years (pre-2006) with low release counts. In addition to this frequency analysis, the average number of released games in a given year is computed.

In [ ]:
# Create a new series of the release dates, with dates reduced to year only.
release_year_freq = (games['release_date']
                     .dt.year
                     .value_counts())

# Plot the release year frequency using Pandas
release_year_freq.sort_index().plot(kind='bar', 
                                    title="Release Year Frequency", 
                                    logy=True, xlabel='Release Year', 
                                    ylabel='Number of Games Released')
plt.show()

# Show the frequency of game releases, sorted by number of releases
print("\n Sorted Game Release Frequency by Year")
print(release_year_freq)

# Statistically characterize the release year distribution
print(f"\nThe average year has approximately {release_year_freq.mean():.1f} games released.")

### Game Price EDA:
The goal of this section is to characterize the price distribution of the games statistically and visually, across the dataset. Here, Pandas' `.describe()` method is used to statistically characterize the distribution of the `price` continuous variable. Furthermore, a logarithmic plot is provided to understand the _entire_ distribution due to the existance of a handful of games in the 975-1000 USD range. Additionally, a histogram is provided in the 0-75 USD range to characterize the _heavy_ majority of the distribution, as shown by the logarithmic plot.

In [ ]:
# Statistically descibe the distribution of the price column
print(games['price'].describe())

# Plot the entire frequency of prices using a histogram, syncing bins with xticks
bins_xticks_range = range(0, 1001, 25)
games['price'].plot(kind='hist', 
                    title="Full Logarithmic Frequency Distribution of Game Price", 
                    logy=True, 
                    xlabel="Price in USD($)", 
                    bins=bins_xticks_range, 
                    xticks=bins_xticks_range, 
                    rot=90
                   )
plt.show()

# Plot the reduced frequency of prices using a histogram, syncing bins with xticks
bins_xticks_range_reduced = range(0, 80, 5)
games['price'].plot(kind='hist', 
                    title="Reduced ($0-75 USD) Frequency Distribution of Game Price",  
                    xlabel="Price in USD($)", 
                    bins=bins_xticks_range_reduced, 
                    xticks=bins_xticks_range_reduced, 
                    rot=90,
                    xlim=(0,75)
                   )
plt.show()

### Operating System Offering EDA:
In this section, the operating system offerings of the games in the dataset are analyzed. The counts of games offered on each OS is reported below.

In [ ]:
# Report the counts of each operating systems' games
print(f"Windows has {games['windows'].sum()} games available.")
print(f"Macintosh has {games['mac'].sum()} games available.")
print(f"Linux has {games['linux'].sum()} games available.")

games[['windows', 'mac', 'linux']].sum().plot(kind='bar',
                                              title="Number of Games per OS",
                                              xlabel='Operating System',
                                              ylabel='Number of Games Offered',
                                              rot=0)
plt.show()

### User Review (Positive/Negative) EDA:
In this section, the number of user reviews (both positive and negative) are statistically analyzed. Additionally, positive and negative reviews are plotted in a single figure to offer a side-by side comparison of the two measures. Note here that the use of the `.describe()` method is primarily used to inform plot parameter selection, and the key takeaways of the statistical measures of these features is restated after the plot.

In [ ]:
# Statistically characterize the number of positive reviews
print("Positive Review Statistics:")
print(games['positive'].describe())

# Statistically characterize the number of negative reviews
print("\nNegative Review Statistics:")
print(games['negative'].describe())

# Plot the histograms of positive and negative reviews in a single figure
games.plot(kind='scatter', 
           x='positive', 
           y='negative',
           xlabel='Positive Reviews',
           ylabel='Negative Reviews',
           title='Positive vs. Negative Reviews'
          )
plt.show()

### Bivariate Analysis 1: Average Game Price per Release Year
In this section, the average price of games was compared to the release year using `.groupby()` with the `.mean()` aggregation function. Results were plotted as a bar graph.

In [ ]:
# Create a new column containing the release year as a category
games['release_year'] = (games['release_date'].dt.year).astype('category')

# Group by year, aggregate price average
avg_price_by_year = (games.groupby('release_year', observed=False)['price']
                     .mean()
                     .sort_index())

# Report average price per year
print(avg_price_by_year)

# Plot the results using a bar graph
avg_price_by_year.plot(kind='bar', 
                       title="Average Yearly Game Price", 
                       logy=True, 
                       xlabel='Release Year', 
                       ylabel='Average Game Price (USD)',
                       rot=90.0)

### Bivariate Analysis 2: Operating System Game Releases by Year
In this section, the number of released games per operating system were computed for each release year using the `.groupby()` methd with the `.sum()` aggregation function to count results.

In [ ]:
# Group by year, aggregate count of each OS game releases
os_releases_by_year = (games.groupby('release_year', observed=False)[['windows', 'mac', 'linux']]
                     .sum()
                     .sort_index())

# Report average price per year, flattened
print(os_releases_by_year.reset_index())

In [ ]:
# looking at the precentage positive
#create percentage positive column
games = games.astype({"positive" : "float64", "negative" : "float64"})
games.info()
games["percent_positive"] = games["positive"] / (games["positive"] + games["negative"])

#report average number of positive and negative reviews and average percentage positive
rec_year = games.groupby("release_year", observed=False)[["positive", "negative", "percent_positive"]].mean()
print(rec_year)

games["percent_positive"].isna().sum()

In [ ]:
# looking at peak ccu
# groupby peack ccu to get mean, sum and count for each year
year_pccu = games.groupby("release_year", observed=False)[["peak_ccu","name"]].agg(
    {"peak_ccu" : ["mean", "sum"],
     "name" : "count"})
print(year_pccu)

#peak ccu stats
print(f'\nPeak CCU Stats:\n{games["peak_ccu"].describe()}')

#tried looking at developers in relation to peak ccu and it needs cleaning i think
#dev_pccu = games.groupby("developers", observed=False)["peak_ccu"].mean().sort_index()




In [ ]:
# language stuff


## option 1 Split and make lists either make new series or change supported languages
#languages =  games["supported_languages"].str.split("/")
#### NEEDS TO BE ADDED EARLIER
#games["supported_languages"] = games["supported_languages"].str.split("/")


## option 2 use str contains and make a series for each interested language
#english = games["supported_languages"].str.contains("English")
#print(english.sum())

## option 3 add a boolean column to for each interested language
games["English"] = games["supported_languages"].str.contains("English")
games["Chinese"] = games["supported_languages"].str.contains("Chinese")
games["Japanese"] = games["supported_languages"].str.contains("Japanese")
games["Spanish"] = games["supported_languages"].str.contains("Spanish")
games["German"] = games["supported_languages"].str.contains("German")
games["French"] = games["supported_languages"].str.contains("French")
games["Russian"] = games["supported_languages"].str.contains("Russian")




In [ ]:
#print(games[["English","Chinese", "Japanese", "Spanish", "German", "French", "Russian"]].sum())

lang = ["English","Chinese", "Japanese", "Spanish", "German", "French", "Russian"]
for x in lang:
    print(f'Language: {x}')
    print(f'Total {x} games: {games[x].sum()}')
    print(f'Average {x} games price: {games["price"].loc[games[x] == True].mean()}')
    print(f'Average {x} games peak ccu: {games["peak_ccu"].loc[games[x] == True].mean()}')
    print(f'Average {x} games positive reviews: {games["positive"].loc[games[x] == True].mean()}')
    print(f'Average {x} games negative reviews: {games["negative"].loc[games[x] == True].mean()}\n')



In [ ]:
# testing developer

temp = games["publishers"].isin(games["developers"])
print(temp.sum())
games["dev_pub"] = games["publishers"].isin(games["developers"])

group_price = games.groupby("dev_pub")[["price", "name"]].agg({
    "price" : ["mean", "sum"],
    "name" : "count"})
print(group_price)

### Section 5: Transform
TODO: 

Note: These are just suggestions, the key detail is that 3-4 features are engineered and analyzed.

1. Feature comparing peak_ccu to the number of supported languages
2. Feature relating num recent reviews to peak ccu (called something like review rate)
3. Feature for does publisher match developer, then analyze how this affects price (yes v no to see if having different costs more)
4. Feature for something like overall value something like price compared to pct positive reviews.
Note: You can unhide and type any of the playtime/pct pos reviews/etc. data points in the cells above that you may decide you need for additional feature engineering i.e. remove that col name from the removal list and then give it a float type in the games.astype() block statement.

### Section 6: Save and Document
In this section, the resulting cleaned dataset with additional engineered features is exported as pickle (for Python users) and CSV (for compatability). Note that these files are ignored by Git for tracking since Git is for source code and not exports.

### Exporting the Cleaned and Feature Engineered Dataset:

In [ ]:
# Reset the index of the dataset to export the AppID along with the rest of the data.
games.reset_index()

# Export as a Pickle file
games.to_pickle("exports/games_cleaned_added_features.pkl")

# Export as CSV, removing index since after resetting the index, the index is a simple integer value
games.to_csv("exports/games_cleaned_added_features.csv", index=False)